In [165]:
%run 'proof.ipynb'

In [57]:
import numpy as np
import random
import pandas as pd

In [58]:
NUM_PLAYERS = 3
PLAYERS = [i for i in range(1, NUM_PLAYERS+1)]
random.seed(7)  # for reproducible output

# Simulating for one initial state

In [59]:
def randSelectPlayer(candidates, exclude=None):
    """
    Randomly selects and returns one entry from the list of candidates.
    If `exclude` is specified, that entry is excluded from the list of candidates.
    """
    choices = list(candidates)
    if exclude:
        choices.remove(exclude)
    return random.choice(choices)

In [65]:
def give(playerWealth):
    """
    Simulates the give action:
    The minimum of the giver's and receiver's wealth is transferred from the giver to the receiver.
    Returns the updated `playerWealth` list.
    """
    # randomly select a giver and receiver
    giver = randSelectPlayer(PLAYERS) 
    receiver = randSelectPlayer(PLAYERS, exclude=giver)
    
    v = playerWealth[giver-1]
    w = playerWealth[receiver-1]
    amt = min(v, w)
    playerWealth[receiver-1] += amt
    playerWealth[giver-1] -= amt
    
    return playerWealth, amt, giver, receiver

In [91]:
def loser(playerWealth):
    """
    Returns the loser (None if no one has lost yet).
    """
    for i in range(NUM_PLAYERS):
        if playerWealth[i] == 0:
            return i+1
    return None

def reportGameProgress(n, giver, receiver, transferAmt, playerWealth):
    """
    Outputs the game progress.
    - n: an integer representing the number of rounds played so far.
    - giver: the player that is chosen to give.
    - receiver: the player that is chosen to receive.
    - transferAmt: an integer representing the amount of money transferred from the giver to the receiver.
    - playerWealth: a dictionary of player:wealth key-value pairs, representing each player's current wealth.
    """
    print(f"Round {n}:")
    print(f"P{giver} gave P{receiver} ${transferAmt}")
    print(f"Current stacks: {playerWealth}\n")

In [116]:
def playGame(x,y,z, printProgress=False):
    """
    Plays one game.
    - x, y, z: integers representing the initial wealth of Player 1, 2, 3, respectively.
    - printProgress: if True, prints the game progress.
    Returns the first loser and number of rounds until the game ends.
    """
    # initialise
    stacks = [x,y,z]
    n = 0
    while True:
        stacks, transferAmt, g, r = give(stacks)
        n += 1
        if printProgress:
            reportGameProgress(n, g, r, transferAmt, stacks)
            
        l = loser(stacks)
        if l:
            return n, l
    # should not reach this point
    return n, -1

In [163]:
from collections import defaultdict

def h_est(data, x,y,z):
    """
    Returns an estimate of h(n,x,y,z) = Pr(player 1 is eliminated in exactly n rounds), 
    using the game data
    """
    data.sort()
    numGames = len(data)
    hxyz, hyxz = defaultdict(int), defaultdict(int)
    for n, l in data:
        hxyz[n] += (l == 1)
        hyxz[n] += (l == 2)
    
    return {k : v/numGames for k,v in hxyz.items()}, {k : v/numGames for k,v in hyxz.items()}

In [192]:
def f_est(data, x,y,z):
    hxyz, hyxz = h_est(data, x,y,z)
    fxyz = {}
    maxn = max(max(hxyz), max(hyxz))
    
    def dh(n):
        return hxyz.get(n, 0) - hyxz.get(n, 0)
    
    @memoized
    def sumdh(n): 
        return sum([dh(i) for i in range(1,n+1)])
    
    for m in range(2, maxn+1):
        fxyz[m] = sumdh(m) - thresh(m)
        
    return fxyz

In [193]:
from random import randint
def rand_init(V=(lambda x,y,z: x<y and y<z), maxVal=100, maxAttempts=100):
    for i in range(maxAttempts):
        rand = [randint(1, maxVal) for _ in range(NUM_PLAYERS)]
        if V(*rand):
            return rand
    return None

In [196]:
def sim_xyz(n=100000, stat=f_est):
    xyz = rand_init()
    return stat([playGame(*xyz, printProgress=False) for _ in range(n)], *xyz)

In [198]:
sim_xyz(stat=h_est)

({1: 0.33391,
  2: 0.02791,
  3: 0.05085,
  4: 0.02323,
  5: 0.01257,
  6: 0.0058,
  7: 0.00269,
  8: 0.00134,
  9: 0.00065,
  10: 0.00035,
  11: 0.0002,
  12: 8e-05,
  13: 1e-05,
  14: 1e-05,
  15: 1e-05,
  16: 0.0,
  19: 0.0},
 {1: 0.16502,
  2: 0.08296,
  3: 0.04077,
  4: 0.02494,
  5: 0.0108,
  6: 0.00594,
  7: 0.00324,
  8: 0.00152,
  9: 0.00062,
  10: 0.00023,
  11: 0.00011,
  12: 3e-05,
  13: 0.0,
  14: 0.0,
  15: 0.0,
  16: 1e-05,
  19: 0.0})

In [197]:
sim_xyz()

{2: -0.0894,
 3: -0.01702999999999999,
 4: 0.028800000000000006,
 5: 0.049810000000000014,
 6: 0.062350000000000017,
 7: 0.06828000000000001,
 8: 0.07122500000000001,
 9: 0.07263750000000002,
 10: 0.07346875000000001,
 11: 0.07395937500000001,
 12: 0.07415468750000001,
 13: 0.07429234375000002,
 14: 0.07435117187500001,
 15: 0.0743555859375,
 16: 0.07437779296875,
 17: 0.074383896484375,
 18: 0.0743869482421875,
 19: 0.07439847412109374,
 20: 0.07439923706054687}

# Simulating the game multiple times and estimating probabilities

In [200]:
from statistics import mean

def sim(n=100000):
    """
    Simulates the betting game `n` times, with different random initial stacks.
    Returns the average f(n,x,y,z)
    """   
    return mean([sim_xyz(stat=f_est) for _ in range(n)])

In [ ]:
sim()